In [1]:
import numpy as np
import pandas as pd
from py2neo import Graph, Node, Relationship

import requests
from bs4 import BeautifulSoup
import os

In [2]:
url = 'https://www.basketball-reference.com/leagues/NBA_2020_per_game.html'
page = requests.get(url)
page.content[13200:13300]

b'-size:.85em;word-spacing:0;white-space:nowrap;overflow:hidden;text-overflow:ellipsis;border-radius:5'

In [6]:
soup = BeautifulSoup(page.content, 'html.parser')

table = soup.find_all(class_="full_table")
head = soup.find_all(class_="thead")

In [12]:
col_names = [_.text for _ in head][0]
col_names = col_names.split('\n')[2:-1]

In [24]:
players = []
for i in range(len(table)):
    player_ = []

    for td in table[i].find_all("td"):
        player_.append(td.text)

    players.append(player_)

df = pd.DataFrame(players, columns=col_names)

In [25]:
df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Steven Adams,C,26,OKC,63,63,26.7,4.5,7.6,.592,...,.582,3.3,6.0,9.3,2.3,0.8,1.1,1.5,1.9,10.9
1,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,11.0,.557,...,.691,2.4,7.8,10.2,5.1,1.1,1.3,2.8,2.5,15.9
2,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,15.0,.493,...,.827,1.9,5.5,7.4,2.4,0.7,1.6,1.4,2.4,18.9
3,Kyle Alexander,C,23,MIA,2,0,6.5,0.5,1.0,.500,...,,1.0,0.5,1.5,0.0,0.0,0.0,0.5,0.5,1.0
4,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,5.7,.368,...,.676,0.2,1.6,1.8,1.9,0.4,0.2,1.1,1.2,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,Trae Young,PG,21,ATL,60,60,35.3,9.1,20.8,.437,...,.860,0.5,3.7,4.3,9.3,1.1,0.1,4.8,1.7,29.6
525,Cody Zeller,C,27,CHO,58,39,23.1,4.3,8.3,.524,...,.682,2.8,4.3,7.1,1.5,0.7,0.4,1.3,2.4,11.1
526,Tyler Zeller,C,30,SAS,2,0,2.0,0.5,2.0,.250,...,,1.5,0.5,2.0,0.0,0.0,0.0,0.0,0.0,1.0
527,Ante Žižić,C,23,CLE,22,0,10.0,1.9,3.3,.569,...,.737,0.8,2.2,3.0,0.3,0.3,0.2,0.5,1.2,4.4


In [22]:
driver = Graph("bolt://localhost:7687", auth=('neo4j', '1234'))

In [26]:
# create nodes

def createPlayerNode(teamNode, **kwargs):
    n = Node("PLAYER", **kwargs)
    n.__primarykey__='Name'
    driver.merge(n)

    playsFor = Relationship("PLAYS_FOR")
    rel = playsFor(n, teamNode)

def createTeamNode(**kwargs):
    n = Node("TEAM", **kwargs)
    n.__primarykey__='Name'
    driver.merge(n)